
### **Supervisor** is a multi-agent architecture where specialized agents are coordinated by a central supervisor agent. The supervisor agent controls all communication flow and task delegation, making decisions about which agent to invoke based on the current context and task requirements.

![supervisor.png](./images/supervisor.png "supervisor.png")

# 🤔 Why not One Agent With All Tools?

Sometimes instead of building a multi-agent system, you can just give **one single agent** access to all tools. This has pros and cons.

---

## 🔴 Drawbacks / Limits

- **Context overload**  
  The agent has to “know” about every tool, its parameters, and when to use it — more instructions in the prompt → higher token usage and possible confusion.

- **Specialization loss**  
  You can’t easily tune or optimize behavior per task. All tasks share the same prompt, temperature, reasoning style, etc.

- **Harder to scale**  
  If you add many more tools or domains (e.g., research, coding, data engineering, finance), the complexity grows linearly for the agent prompt, but its reasoning reliability often drops.

- **No task decomposition**  
  Unless you explicitly teach it how to break tasks into subtasks, it will try to do everything in one pass — which can be less reliable for multi-step work.

- **Less observability & control**  
  In a multi-agent system, you can inspect each step, retry parts, or parallelize work. With one big agent, everything is opaque in one “thought process.”

---

## 🧠 Mental Model

Think of it like this:

| Setup                   | Analogy                   | When good                                                                |
|-------------------------|---------------------------|--------------------------------------------------------------------------|
| One agent with all tools | Swiss Army knife           | Few tools, moderate complexity, want simplicity                          |
| Multi-agent supervisor   | Project manager with experts | Many domains, complex workflows, need reliability, scalability, and transparency |



### Create Multi-agent supervisor using LangGraph

# Multi-Agent System Steps

1. **Build specialized research and math agents**  
   Create individual agents focused on specific domains (e.g., one for research, one for mathematical reasoning).

2. **Build a supervisor for orchestrating them with the prebuilt `langgraph-supervisor`**  
   Use the prebuilt LangGraph Supervisor to manage task routing and delegation between agents.

3. **Build a supervisor from scratch**  
   Implement a custom supervisor to control task flow and agent collaboration manually.

4. **Implement advanced task delegation**  
   Add logic for dynamic task prioritization, parallel execution, or conditional delegation to optimize performance.


In [0]:
import os

os.environ['TAVILY_API_KEY'] = "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"

In [0]:
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=3)
web_search_results = web_search.invoke("who is the mayor of NYC?")

print(web_search_results["results"][0]["content"])



### Research agent



In [0]:
from langgraph.prebuilt import create_react_agent

from databricks_langchain import (
    ChatDatabricksx
)

LLM_ENDPOINT_NAME = "databricks-gpt-oss-120b"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

research_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)

In [0]:
from helpers import pretty_print_messages
for chunk in research_agent.stream(
    {"messages": [{"role": "user", "content": "who is the mayor of NYC?"}]}
):
    pretty_print_messages(chunk)

In [0]:
def add(a: float, b: float):
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b


math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply, divide],
    prompt=(
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="math_agent",
)

In [0]:
for chunk in math_agent.stream(
    {"messages": [{"role": "user", "content": "what's (3 + 5) x 7"}]}
):
    pretty_print_messages(chunk)

In [0]:
from langgraph_supervisor import create_supervisor

supervisor = create_supervisor(
    model=llm,
    agents=[research_agent, math_agent],
    prompt=(
        "You are a supervisor managing two agents:\n"
        "- a research agent. Assign research-related tasks to this agent\n"
        "- a math agent. Assign math-related tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()

In [0]:
from IPython.display import display, Image

display(Image(supervisor.get_graph().draw_mermaid_png()))

In [0]:
for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "find US and New York state GDP in 2024. what % of US GDP was New York state?",
            }
        ]
    },
):
    pretty_print_messages(chunk, last_message=True)

final_message_history = chunk["supervisor"]["messages"]

In [0]:
import json
from typing import Any, Generator
from uuid import uuid4

import mlflow
from databricks_langchain import ChatDatabricks, UCFunctionToolkit
from langchain_core.messages import AIMessageChunk
from langgraph.prebuilt import create_react_agent

from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)


In [0]:
############################################
# Register the agent with MLflow
############################################
from helpers import LangGraphResponsesAgent
mlflow.langchain.autolog()
AGENT = LangGraphResponsesAgent(supervisor)
mlflow.models.set_model(AGENT)

In [0]:
result = AGENT.predict({"input": [{"role": "user", "content": "find US and New York state GDP in 2024. what % of US GDP was New York state?"}]})
print(result.model_dump(exclude_none=True))